### Objective
I will cleaning and exploring this dataset using pandas. It will be visualised using matplotlib. Afterwards, I'll hypothesise the relationship and trends of this data. This dataset is obtained via [kaggle.com](https://www.kaggle.com/datasets/bharatnatrayn/movies-dataset-for-feature-extracion-prediction/data).


#### Importing Libraries

In [799]:
import pandas as pd
import numpy as np
import re
from IPython.display import display
import matplotlib as plt

#### Import Raw Dataset

In [800]:
df = pd.read_csv('Data/raw_movies.csv')
display(df.head())

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\r\nAction, Horror, Thriller",6.1,\r\nA woman with a mysterious illness is force...,\r\n Director:\r\nPeter Thorwarth\r\n| \r\n...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\r\nAnimation, Action, Adventure",5.0,\r\nThe war for Eternia begins again in what m...,"\r\n \r\n Stars:\r\nChris Wood, ...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\r\nDrama, Horror, Thriller",8.2,\r\nSheriff Deputy Rick Grimes wakes up from a...,\r\n \r\n Stars:\r\nAndrew Linco...,"885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\r\nAnimation, Adventure, Comedy",9.2,\r\nAn animated series that follows the exploi...,\r\n \r\n Stars:\r\nJustin Roila...,"414,849",23.0,NaN
4,Army of Thieves,(2021),"\r\nAction, Crime, Horror",NaN,"\r\nA prequel, set before the events of Army o...",\r\n Director:\r\nMatthias Schweighöfer\r\n...,NaN,NaN,NaN


### Data Inspection

In [801]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


(9999, 9)

In [802]:
df.describe()

,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [803]:
df.isnull().sum()

MOVIES         0
YEAR         644
GENRE         80
RATING      1820
ONE-LINE       0
STARS          0
VOTES       1820
RunTime     2958
Gross       9539
dtype: int64

#### Observations
- There are 9 columns, with 9999 rows
- Most columns stores object
- RATING, RunTime: float64
- Columns w/ NULL: YEAR, GENRE, RATING, VOTES, RunTime, Gross

### Renaming Columns

In [804]:
df_col = df.rename(columns=
          {'MOVIES': 'Movies', 'YEAR': 'Year',
           'GENRE': 'Genre', 'RATING': 'Rating',
           'ONE-LINE': 'Short Desc', 'STARS': 'Stars',
           'VOTES': 'Votes', 'RunTime': 'Run Time',
           'Gross': 'Gross'})
print(df_col.columns)

Index(['Movies', 'Year', 'Genre', 'Rating', 'Short Desc', 'Stars', 'Votes',
       'Run Time', 'Gross'],
      dtype='object')


### Removing Duplicates

In [805]:
print(f'Duplicates: {df_col.duplicated().sum()}')
df_dupe = df_col.drop_duplicates()

Duplicates: 431


### Fixing Data Types
Votes: Remove ',' - convert to integer.

Gross: Remove '$' and 'M' - convert to float.

In [806]:
df_dupe.loc[:, 'Votes'] = df_dupe['Votes'].str.replace(',', '').apply(lambda x: int(x) if x is not np.nan else x)
df_dupe.loc[:, 'Gross'] = df_dupe['Gross'].str.replace('$', '').str.replace('M', '')

In [807]:
df_dupe = df_dupe.astype({'Movies': 'object',
                          'Year': 'object',
                          'Genre': 'object',
                          'Rating': 'float64',
                          'Short Desc': 'object',
                          'Stars': 'object',
                          'Votes': 'Int64',
                          'Run Time': 'float64',
                          'Gross': 'float64'})

In [808]:
df_fix = df_dupe.copy()
df_fix.info()

filtered_df_fix = df_fix[df_fix['Gross'].notnull()]
display(df_fix.head(3))

<class 'pandas.core.frame.DataFrame'>
Index: 9568 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9568 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       9490 non-null   object 
 3   Rating      8168 non-null   float64
 4   Short Desc  9568 non-null   object 
 5   Stars       9568 non-null   object 
 6   Votes       8168 non-null   Int64  
 7   Run Time    7008 non-null   float64
 8   Gross       460 non-null    float64
dtypes: Int64(1), float64(3), object(5)
memory usage: 756.8+ KB


,Movies,Year,Genre,Rating,Short Desc,Stars,Votes,Run Time,Gross
0,Blood Red Sky,(2021),"\r\nAction, Horror, Thriller",6.1,\r\nA woman with a mysterious illness is force...,\r\n Director:\r\nPeter Thorwarth\r\n| \r\n...,21062,121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\r\nAnimation, Action, Adventure",5.0,\r\nThe war for Eternia begins again in what m...,"\r\n \r\n Stars:\r\nChris Wood, ...",17870,25.0,NaN
2,The Walking Dead,(2010–2022),"\r\nDrama, Horror, Thriller",8.2,\r\nSheriff Deputy Rick Grimes wakes up from a...,\r\n \r\n Stars:\r\nAndrew Linco...,885805,44.0,NaN


### Handling Missing Values
Gross & Run Time are the least important to the average viewers. Hence, the rows with missing values for any of these columns will not be removed.

Missing Values for Ratings & Votes will be replaced by the mean average for all recorded movies in the dataset.

All rows with missing Year will be removed.

In [809]:
# Drop rows with missing Year
df_mis = df_fix.dropna(subset=['Year'])

# Fill missing Ratings and Votes with mean
df_mis.loc[:, 'Rating'] = df_mis['Rating'].fillna(df_mis['Rating'].mean())
df_mis.loc[:, 'Votes'] = df_mis['Votes'].fillna(round(df_mis['Votes'].mean()))

In [810]:
df_mis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9026 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       8989 non-null   object 
 3   Rating      9026 non-null   float64
 4   Short Desc  9026 non-null   object 
 5   Stars       9026 non-null   object 
 6   Votes       9026 non-null   Int64  
 7   Run Time    6987 non-null   float64
 8   Gross       460 non-null    float64
dtypes: Int64(1), float64(3), object(5)
memory usage: 714.0+ KB


### Formatting Text Data
Stars: Format from strings into dictionary:
- 'Director' & 'Stars' are keys.
- Names are stored in an array.

Example: {'Director': ['Person A', ..., 'Person N'], 'Stars': ['Person C', ... , 'Person M']}

- Genre: Remove \r\n - convert into an array.
- Short Desc: Remove \r\n.
- Rating: Round to 2 decimal place.
- Year: Convert into a tuple (StartYear, EndYear)
- Split Stars column into two: Director & Stars

In [811]:
def clean_genre(text):
    if type(text) == str:
        return text.strip().replace('\r\n', '').split(',')

In [812]:
def clean_short_desc(text):
    return text.strip().replace('\r\n', '')

In [813]:
def clean_rating(val):
    return round(val, 2)

In [814]:
def clean_year(val):
    val = val.replace('(', '').replace(')', '')
    for i in val:
        if not i.isdigit() and i != '–':
            val = val.replace(i, '')
            
    if '–' not in val:
        return [val]
    arr = val.split('–')
    arr = [arr[0], None] if arr[1] == '' or arr[1] == ' ' else [arr[0], arr[1]]
    
    arr = [int(x) if x is not None else x for x in arr]
    return arr

In [815]:
df_mis.loc[:, 'Year'] = df_mis['Year'].apply(clean_year)

In [816]:
df_mis.loc[:, 'Genre'] = df_mis['Genre'].apply(clean_genre)

In [817]:
df_mis.loc[:, 'Short Desc'] = df_mis['Short Desc'].apply(clean_short_desc)

In [818]:
df_mis.loc[:, 'Rating'] = df_mis['Rating'].apply(clean_rating)

In [819]:
def clean_stars_old(text):
    # Initialize dictionary
    dict = {'Director': [], 'Stars': []}
    
    # Remove whitespaces & labels (Director, Stars)
    removed_whitespace = text.strip().replace('\r\n', '').replace(' ', '')
    removed_labels = removed_whitespace.replace('Director:', '').replace('Stars:', '')
    
    # Split Director and Stars
    split_labels = removed_labels.split('|')
    
    # Split each element into an array
    if len(split_labels) == 2:
        dict.update({'Director': split_labels[0], 'Stars': split_labels[1]})
        for i in dict:
            dict[i] = dict[i].split(',')
    else:
        dict.update({'Director': split_labels[0], 'Stars': None})
    
        
    return dict

In [820]:
def clean_stars(text):
    # Initialize dictionary
    dict = {'Director': [], 'Stars': []}
    
    # Remove whitespaces
    removed_whitespace = text.strip().replace('\r\n', '').replace(' ', '')
    
    # Split Director and Stars
    if '|' in removed_whitespace:
        removed_labels = removed_whitespace.replace('Director:', '').replace('Stars:', '')
        split_labels = removed_labels.split('|')
        
        # Split each element into an array
        if len(split_labels) == 2:
            dict.update({'Director': split_labels[0], 'Stars': split_labels[1]})
            for i in dict:
                dict[i] = dict[i].split(',')
        else:
            dict.update({'Director': split_labels[0], 'Stars': np.nan})
    
    # If there is only Directors    
    elif 'Stars:' not in removed_whitespace:
        removed_labels = removed_whitespace.replace('Director:', '')
        split_labels = removed_labels.split(',')        
        dict.update({'Director': split_labels, 'Stars': np.nan})
    
    # If there is only Stars    
    elif 'Director:' not in removed_whitespace:
        removed_labels = removed_whitespace.replace('Stars:', '')
        split_labels = removed_labels.split(',')
        dict.update({'Director': np.nan, 'Stars': split_labels})
                
    # Format the first and last name           
    for key, val in dict.items():
        if isinstance(val, list):
            dict[key] = [re.sub(r'(?<!^)(?=[A-Z])', ' ', name) if isinstance(name, str) and not pd.isna(name) else name for name in val]
        elif isinstance(val, str) and not pd.isna(val):
            dict[key] = re.sub(r'(?<!^)(?=[A-Z])', ' ', val)
                    
    return dict

In [821]:
def create_director_list(text):
    dict = clean_stars(text)
    return dict['Director']

In [822]:
def create_star_list(text):
    dict = clean_stars(text)
    return dict['Stars']

In [823]:
# {'Director': ['Person A', ..., 'Person N'], 'Stars': ['Person C', ... , 'Person M']}
df_mis.loc[:, 'Directors'] = df_mis['Stars'].apply(create_director_list)
df_mis.loc[:, 'Stars'] = df_mis['Stars'].apply(create_star_list)

C:\Users\frasid\AppData\Local\Temp\ipykernel_17500\2775121614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mis.loc[:, 'Directors'] = df_mis['Stars'].apply(create_director_list)


In [824]:
df_format = df_mis.copy()
df_format = df_format[['Movies', 'Year', 'Genre', 'Rating', 'Short Desc', 'Votes', 'Run Time', 'Gross', 'Directors', 'Stars']]
display(df_format.head())
df_format.info()

,Movies,Year,Genre,Rating,Short Desc,Votes,Run Time,Gross,Directors,Stars
0,Blood Red Sky,[2021],"[Action, Horror, Thriller]",6.10,A woman with a mysterious illness is forced in...,21062,121.0,NaN,[Peter Thorwarth],"[Peri Baumeister, Carl Anton Koch, Alexander S..."
1,Masters of the Universe: Revelation,"[2021, None]","[Animation, Action, Adventure]",5.00,The war for Eternia begins again in what may b...,17870,25.0,NaN,NaN,"[Chris Wood, Sarah Michelle Gellar, Lena Heade..."
2,The Walking Dead,"[2010, 2022]","[Drama, Horror, Thriller]",8.20,Sheriff Deputy Rick Grimes wakes up from a com...,885805,44.0,NaN,NaN,"[Andrew Lincoln, Norman Reedus, Melissa Mc Bri..."
3,Rick and Morty,"[2013, None]","[Animation, Adventure, Comedy]",9.20,An animated series that follows the exploits o...,414849,23.0,NaN,NaN,"[Justin Roiland, Chris Parnell, Spencer Gramme..."
4,Army of Thieves,[2021],"[Action, Crime, Horror]",6.92,"A prequel, set before the events of Army of th...",15144,NaN,NaN,[Matthias Schweighöfer],"[Matthias Schweighöfer, Nathalie Emmanuel, Rub..."


<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 9998
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9026 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       8989 non-null   object 
 3   Rating      9026 non-null   float64
 4   Short Desc  9026 non-null   object 
 5   Votes       9026 non-null   Int64  
 6   Run Time    6987 non-null   float64
 7   Gross       460 non-null    float64
 8   Directors   6284 non-null   object 
 9   Stars       8511 non-null   object 
dtypes: Int64(1), float64(3), object(6)
memory usage: 784.5+ KB


### Handling Outliers

### Filtering Irrelevant Data

### Validating Data Consistency